In [ ]:
# https://dataspace.copernicus.eu/news/2023-9-28-accessing-sentinel-mission-data-new-copernicus-data-space-ecosystem-apis

In [ ]:
import os
import geopandas as gpd
from sentinelsat import SentinelAPI, make_path_filter
from IPython import display # lets us display images that we download
import shapely
import requests
import netrc

In [ ]:
url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

# Specify the path to the .netrc file
netrc_file_path = r"C:\Users\kinse\.netrc.txt"

# Load credentials from .netrc file
access_token = netrc.netrc(netrc_file_path).authenticators("identity.dataspace.copernicus.eu")

if access_token:
    username, _, password = access_token
    data = {
        "username": username,
        "password": password,
        "grant_type": "password",
        "client_id": "cdse-public"
    }

    response = requests.post(url, headers=headers, data=data)
    print(response.json())
else:
    print("Credentials not found in .netrc file.")

In [ ]:
start_date = "2022-06-01"
end_date = "2022-06-10"
data_collection = "SENTINEL-2"
aoi = "POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))'"


In [ ]:
json = requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}) and ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T00:00:00.000Z").json()
gpd.GeoDataFrame.from_dict(json['value']).head(5)


In [ ]:
url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products(acdd7b9a-a5d4-5d10-9ac8-554623b8a0c9)/$value"

headers = {"Authorization": f"Bearer {access_token}"}

session = requests.Session()
session.headers.update(headers)
response = session.get(url, headers=headers, stream=True)

with open("product.zip", "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)
            